# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1", "abAB", "cdCD", "efEF"]


["1", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Random Word generator

In [5]:
def word_generator(length=5)
  rstr = ''
  length.times{|k| rstr += Rseq.sample.to_s}
  return Word.new(rstr)
end
def wordgen(length=5)
  word_generator(length)
end;nil

# Goldman bracket & Turaev cobracket

## Partition class, Term class and Expansion class

In [6]:
class Partition < Hash
  def initialize(word: Word.new, div: 0)
    self[:word] = word
    self[:div] = div
  end
end

#----------------------------------
class Term < Hash
  '''
    A Hash with keys :coeff, :words and :divs
  '''
  def initialize(coeff: 0, words: [], divs: [])
    begin      
      self[:coeff] = coeff
      self[:words] = words
    rescue => e
      p e.backtrace
    end
  end
  
  def cyclic_reduce
    trm = self.class.new(coeff: self[:coeff], divs: self[:divs])
    trm[:words] = self[:words].map(&:cyclic_reduce)
    return trm
  end
  
  def show
    coeff = (self[:coeff] == 1) ? '' : "(#{self[:coeff]})"
    body = self[:words].map(&:show).join("\u{2227}")
    coeff + body
  end
  
  def equiv?(a_term)
    '''
      return 1(match), -1(reversely match) and 0(not match).
    '''
    (self[:words].size).times do |k|
      flags = self[:words].zip(a_term[:words].rotate(k)).map{|pair| pair[0].is_cyclically_same?(pair[1])}
      if flags.all?{|tf| tf}
        return (-1)**k
        break
      end
    end
    return 0
  end
end

#-----------------------------------
class Expansion < Array
  '''
    An Array of Terms
  '''
  def *(int)
    raise ArgumentError, int.class unless int.is_a?(Integer)
    return self.each_with_object(self.class.new) do |term, expn|
      term[:coeff] *= int
      expn << term
    end
  end
#  def +(another)  
#    raise ArgumentError, another.class unless another.is_a?(self.class)
#    return (self.concat(another)).each_with_object(self.class.new){|term, expn| expn << term}
#   #NOTE: Using the parent's method '+', (self + another) yields an Array.class object.
#  end
  def show(simplify_level=0)
    expn = case simplify_level
      when 1
        self.delete_if{|t| t[:coeff] == 0}
      when 2
        self.simplify
      else 
        self.rotate(0)
      end
    #---
    mstr = expn.map{|term| term.show}.join(' + ')
    return (mstr.empty?) ? '0' : mstr
  end

  def simplify
    expn = self.class.new
    monomials = self.map{|mono| mono.cyclic_reduce unless mono[:coeff] == 0}.compact
    until monomials.size == 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        switch = mono.equiv?(pop)
        unless switch == 0
          pop[:coeff] += mono[:coeff] * switch
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      expn << pop if pop[:coeff] != 0
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    return (expn.empty?) ? (self.class.new << Term.new) : expn
  end
end; nil

### scratch

In [7]:
w = Word.new('abc').cyclic_permutation
p w.class
nil

Word


In [8]:
terms = [
  Term.new,
  Term.new(words: [wordgen(8)], coeff: -1), 
  Term.new(words: [wordgen(3), wordgen(2)], coeff: 1)
]

terms.each{|t| printf t.show + ' |-reduce-> ' + t.cyclic_reduce.show + "\n" }
nil

(0) |-reduce-> (0)
(-1)fDCdADEF |-reduce-> (-1)DCdADE
FfA∧Ab |-reduce-> A∧Ab


In [9]:
count = 10
count.times do |k|
  expn = Expansion.new() 
  5.times do |i|
    cf = ( (-1)**(i.modulo(3)) - (-1)**((i+1).modulo(3)) )/2
    ws = [3,3].map{|k| word_generator(k)}
    expn << Term.new(divs: '', coeff: cf, words: ws)
  end
  if expn.simplify.size < 4
    break
  end
end; nil

## Linking of a pair of divisions

In [10]:
def lk(ptn1, ptn2)
  '''
    Input: Patitions {word: a Word, div: an pos. Integer}
  '''
  hashes = [ptn1, ptn2].map do |ptn|
    w = ptn[:word].flatten
    k = ptn[:div]
    s = w.size
    [
      {index: ( k%s ) + 1, letter: w[ k%s ], sign: 1}, 
      {index: k, letter: w[k-1].inverse, sign: -1}
    ]
  end  
  total = [0,1].product([0,1]).map{|i,j| epsilon(hashes[0][i], hashes[1][j])}.sum
  return (-1)*(total/2)
end

#--------------------------------------------
def epsilon(h1, h2)
  '''
    Input: Hashes {index: a pos. Integer, letter: a Letter, sign: 1 or -1}
  '''
  result = h1[:sign]*h2[:sign]
  result *= unless (h1[:letter].show == h2[:letter].show)
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  else
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  end
  return result
end; nil

## Goldman bracket

In [11]:
def bracket(w1, w2)
  """
  retern: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  #-----
  expn = Expansion.new
  unless (w1.show == '1' || w2.show == '1')
    ws = [w1, w2]
    (w1.size.times.to_a).product(w2.size.times.to_a).each do |inds|
      #--- the pair of partitions ---
      ptns = [0,1].map{|k| Partition.new(word: ws[k], div: inds[k]+1)}
      #--- put the two words togather ---
      cpws = [0,1].map{|k| ws[k].cyclic_permutation(inds[k]+1)}
      expn << Term.new(divs: ptns.map{|p| p[:div]}, coeff: lk(*ptns), words: [cpws[0]*cpws[1]])
    end
  end
  return expn
end; nil

### scratch

In [12]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
w1, w2 = wordgen(2), wordgen(3)
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

bra.each do |t|
  t.each{|k,v| printf "#{k}: " + ((k == :words) ? v[0].show : v.to_s) + ", "}
  printf "\n---\n"
end
puts bra.show, "= " + bra.show(2)
nil

cb, dfB
-------
coeff: 1, words: bcfBd, 
---
coeff: 0, words: bcBdf, 
---
coeff: 0, words: bcdfB, 
---
coeff: 0, words: cbfBd, 
---
coeff: 1, words: cbBdf, 
---
coeff: -1, words: cbdfB, 
---
bcfBd + (0)bcBdf + (0)bcdfB + (0)cbfBd + cbBdf + (-1)cbdfB
= (-1)cbdfB + cdf + bcfBd


## Turaev cobraket

In [13]:
def cobracket(myw, verbose: false)
  """
  return: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless myw.show == '1' 
    ptn_num_pairs = (1..myw.size).to_a.combination(2)  #all the pair of partition numbers
    ptn_num_pairs.each do |i,j|
      #--- the pair of Partitions ---
      ptns = [i,j].map{|k| Partition.new(word: myw, div: k)} 
      #--- divide the word --- 
      words = myw.cyclic_permutation(i).split(j-i).reverse
      expn << Term.new(divs: [i,j], coeff: lk(*ptns), words: words)
    end
  end
  #-----
  return expn
end; nil

### scratch

In [14]:
#mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
mw = wordgen(5)
printf mw.show + "\n-------\n"

cobra = cobracket(mw, verbose: false) #true) #

# cobra.each do |t|
#   t.each do |k,v| 
#     p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)
#   end
#   printf "---\n"
# end

puts cobra.show, "= " + cobra.show(2)
nil

daebE
-------
ebEd∧a + (-1)bEd∧ae + Ed∧aeb + (0)d∧aebE + bEda∧e + (0)Eda∧eb + (0)da∧ebE + (0)Edae∧b + (0)dae∧bE + (0)daeb∧E
= bEda∧e + Ed∧aeb + (-1)bEd∧ae + ebEd∧a


# Conditions

## Jacobi Identity

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [32]:
three_words = Array.new(3)
max_length = 3

three_words.map!{|w| word_generator(rand(1..max_length))}
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> " + "\n------"

total = Expansion.new
3.times do |k|
  bra = bracket(three_words[0],three_words[1]); puts "\t\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
  bra.each do |h|
    bra2 = bracket(h[:words][0], three_words[2]); puts "\t\t+ (#{bra2.show(1)}) \u{2297} #{three_words[2].show}"
    total.concat( bra2 * h[:coeff] )
  end
  three_words.rotate!
end

printf "---------\n"
p total.show(1) + " ="

total.show(2)

B ⊗ Ce ⊗ DCd  |---> 
------
	∇(B ⊗ Ce) ⊗ DCd =
		+ (CBeCdD + (-1)CBedDC) ⊗ DCd
		+ (CeBCdD + (-1)CeBdDC) ⊗ DCd
	∇(Ce ⊗ DCd) ⊗ B =
		+ (0) ⊗ B
		+ (0) ⊗ B
		+ (0) ⊗ B
		+ (0) ⊗ B
		+ (0) ⊗ B
		+ (0) ⊗ B
	∇(DCd ⊗ B) ⊗ Ce =
		+ ((-1)dDBCeC + dDBCCe + BCdDeC + (-1)BCdDCe + (-1)CdDBeC + CdDBCe) ⊗ Ce
		+ ((-1)dDCBeC + dDCBCe) ⊗ Ce
		+ ((-1)dBDCeC + dBDCCe + BDCdeC + (-1)DCdBeC) ⊗ Ce
---------
"0 ="


"(0)"

### scratch

In [16]:
three_words = Array.new(3)
max_length = 3
count = 0 #500 #

count.times do |k|
  three_words.map!{|w| word_generator(rand(1..max_length))}
  total = Expansion.new
  3.times do |k|
    bra = bracket(three_words[0],three_words[1])
#     puts "\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
    bra.each do |h|
      bra2 = bracket(h[:words][0], three_words[2])
#       puts "+ (#{bra2.show}) \u{2297} #{three_words[2].show}"
      bra2.each do |hh|
        hh[:coeff] *= h[:coeff]
        total << hh
      end
    end
    three_words.rotate!
  end
  result = total.show(2)
  puts three_words.map(&:show).join(" \u{2297} ") + "  |--> " + result if result != '(0)'
end; nil

## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [22]:
length = 8
myw = word_generator(length) #Word.new('ffA')
#puts myw.show + "\n======"

cobra = cobracket(myw)
puts "\u{03B4}(#{myw.show}) = " + cobra.show(1)
puts "---"

total = cobra.each_with_object(Expansion.new) do |h, expn|
  bra = bracket(*h[:words]) * h[:coeff]
  expn.concat( bra )
  #--- display ---
  tensor = h[:words].map(&:show).join(" \u{2297} ")
  puts "\t\u{2207}(#{tensor}) = #{bra.show(1)}"
  #---------------
end

puts "-----\n= " + total.show(1) + "  =  "
total.show(2)

δ(acadcEBA) = dcEBAac∧a + EBAac∧adc + (-1)BAac∧adcE + (-1)EBAaca∧dc + (-1)BAacadc∧E
---
	∇(dcEBAac ⊗ a) = BAacdcEa + acdcEBAa + (-1)cdcEBAaa
	∇(EBAac ⊗ adc) = BAacEadc + acEBAadc + (-1)cEBAadca + cEBAacad + EBAacdca + (-1)EBAacadc
	∇(BAac ⊗ adcE) = (-1)acBAadcE + cBAadcEa + (-1)cBAacEad + (-1)BAacdcEa
	∇(EBAaca ⊗ dc) = (-1)caEBAacd
	∇(BAacadc ⊗ E) = 0
-----
= BAacdcEa + acdcEBAa + (-1)cdcEBAaa + BAacEadc + acEBAadc + (-1)cEBAadca + cEBAacad + EBAacdca + (-1)EBAacadc + (-1)acBAadcE + cBAadcEa + (-1)cBAacEad + (-1)BAacdcEa + (-1)caEBAacd  =  


"(0)"

### scratch

In [18]:
count = 100 #10000 #
length = 5 #10
count.times do |k|
  myw = word_generator(length)
  total = cobracket(myw).each_with_object(Expansion.new) do |h, expn|
    expn.concat( bracket(*h[:words]) * h[:coeff] )
  end
  result = total.show(2)
  p myw.show + " |--> " + result if result != "(0)"
end; nil

## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = w\cdot\delta(v)- v\cdot\delta(w),
$$
where 
$$
u\cdot(x\otimes y) = [u,x]\otimes y + x\otimes[u,y].
$$
**Note**: From the last formula, we immediately have the following:
$$
u \cdot(x\wedge y) = [u,x]\wedge y - [u,y]\wedge x.
$$

In [28]:
v, w = word_generator(3), word_generator(5) #samples[:s2][0] #Word.new(a[1], a[2]),samples[:s4][0] #Word.new(b[1])
puts "#{v.show} \u{2297} #{w.show}"

#--- Left-hand eq. -------------------
lhe = Expansion.new
bra = bracket(v, w) #; puts "\u{2207}(#{v}, #{w}) = " + bra.show
bra.each do |trm|
  cobra = cobracket(trm[:words][0]) * trm[:coeff] #; puts "\t(#{trm[:coeff]})\u{03B4}(#{trm[:words][0].show}) = " + cobra.show(1) if trm[:coeff] != 0
  lhe.concat( cobra )
end
printf "\nLeft-hand Eq.:  " + lhe.show(2) + "\n---"

#--- Right-hand eq. -------------------
rhe = Expansion.new
double_idx = [0,1].product([0,1])
double_idx.each do |i,j|
  pair01, sign01 = [v,w].rotate(i), (-1)**i
  #---
  u = pair01[0]
  cobra = cobracket(pair01[1]) * sign01 #; printf "#{u}.\u{03B4}(#{pair01[1].show}) = #{u}.(#{cobra.show(1)})\n---\n"
  cobra.each do |term|
    pair02, sign02 = term[:words].rotate(j), (-1)**j
    #---
    x, y = pair02[0], pair02[1]
    bra = bracket(u, x) * (term[:coeff] * sign02) #; printf "\t[#{u}, #{x}] \u{2297} #{y} \n" + "\t=(#{bra.show(1)}) \u{2297} #{y}"
    bra = bra.each_with_object(Expansion.new) do |mono, expn|
      mono[:words][1] = y
      expn << mono
    end
    rhe.concat( bra )
  end
#   printf "\n---\n"
end
#-----
printf "\nRight-hand Eq.:  " + rhe.show(2)

#-----------------------
"Left - Right = " + (lhe.concat( rhe * (-1) )).show(2)

eaf ⊗ FcFfb

Left-hand Eq.:  (-1)bfeaFc∧1 + bfeaF∧c + cbfea∧F + fe∧aFcb + bFafec∧1 + Fafe∧cb + (-1)bFafe∧c + a∧ecb
---
Right-hand Eq.:  (-1)Fcba∧fe + (-1)cbe∧a + feacb∧F + (-1)feaFcb∧1 + afecbF∧1 + afeF∧cb + feaFb∧c + (-1)afebF∧c

"Left - Right = (0)"

An example whose results of the both side of the compatibility condition are very long.

In [20]:
v,w = Word.new('fbC'), Word.new('CfDBe')
nil

# Experiments

cobracket calclation samples

In [21]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^2,
    a1^3,
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s).show(2) }
end;nil

--- s0 ---
 1  |--δ-->  (0)
a  |--δ-->  (0)
ad  |--δ-->  (-1)a∧d
aD  |--δ-->  (0)
aabAaBA  |--δ-->  1∧a
--- s1 ---
 aa  |--δ-->  a∧a
aaa  |--δ-->  (0)
aaaa  |--δ-->  aa∧aa
abA  |--δ-->  1∧b
abAB  |--δ-->  (0)
bab  |--δ-->  (0)
--- s2 ---
 adcDC  |--δ-->  (-1)a∧dcDC
aaBAdcDCabA  |--δ-->  1∧aBAdcDCab
adcDCcdCDaBAdcDCabA  |--δ-->  (2)1∧aBAdcDCab
aBAdcDCab  |--δ-->  (0)
--- s3 ---
 acdCD  |--δ-->  (0)
acdCDcdCD  |--δ-->  acdCD∧cdCD
acdCDabAcdCDaBA  |--δ-->  (0)
acdCDcdCDdcDCabAcdCDaBA  |--δ-->  (0)
--- s4 ---
 accc  |--δ-->  acc∧c + ac∧cc
acDcDcD  |--δ-->  acDcD∧cD + acD∧cDcD
--- s5 ---
 aceCE  |--δ-->  (-1)aceC∧E + ace∧CE + (-1)ac∧C
acdCDefEFceCEfeFEdcDC  |--δ-->  (-1)feFEdcDCacdCDefEFceC∧E + feFEdcDCacdCDefEFce∧CE + (-1)feFEdcDCacdCDefEFc∧C
--- s6 ---
 abcBC  |--δ-->  (-1)abcB∧C + abc∧BC + (-1)Cab∧cB
aAba  |--δ-->  (0)
abAAa  |--δ-->  1∧b
abAaBA  |--δ-->  (0)
abBd  |--δ-->  (-1)a∧d + (-1)da∧1
--- s7 ---
 aBabaB  |--δ-->  (0)
abaBaB  |--δ-->  (0)
aaBAaB  |--δ-->  aB∧aB
aaBB  |--δ-->  (-1)

# TODO

* [ ] wedge の一方に 1 があるものの扱いをはっきりさせる。
* [ ] $x\wedge x$ をゼロと認識させる。
* [x] Expansion#show のオプションに "係数 0 の項だけ消去" を加えたい。
* [x] Compatibility が全然ダメ。
* [x] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。